# Exploring the Data

In [36]:
#Downloading basic shakespeare dataset
!curl -L -o input.txt https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 1089k  100 1089k    0     0  3129k      0 --:--:-- --:--:-- --:--:-- 3130k


In [37]:
with open("input.txt", 'r', encoding='utf-8') as f:
    text = f.read()

In [38]:
print("Length of text: ", len(text))

Length of text:  1115394


In [39]:

print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [40]:
# Understand characters in data and text
chars = sorted((set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [41]:
#Create str to int mapping and vice-versa to tokenize
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # take in a string and output a list of integers for each char
decode = lambda l: ''.join([itos[i] for i in l]) # take a list of integers and map them to a string


In [42]:
test_str = "Hey there dude!"
encoded_str = encode(test_str)
decoded_str = decode(encoded_str)

In [43]:
print("Encoded Str: ", encoded_str)
print("Decoded Str: ", decoded_str)
if (decoded_str == test_str):
    print("Decoder works")

Encoded Str:  [20, 43, 63, 1, 58, 46, 43, 56, 43, 1, 42, 59, 42, 43, 2]
Decoded Str:  Hey there dude!
Decoder works


In [44]:
import torch

In [45]:

encoded_input = encode(text)
data = torch.tensor(encoded_input, dtype=torch.long)

print(data.shape, data.dtype)
print(data[:100])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59])


In [46]:
# Train test split
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [47]:
#chunking into blocks for training
block_size = 8
train_data[:block_size + 1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

What we'd like is "For every sequence of number (18, (18,47), (18, 47, 57)) the target is what comes after the sequence"

In [48]:
x = train_data[:block_size] # input block
y = train_data[1:block_size+1] # targets will be characters to the right of the intial character

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"When context is {context}, target is {target}")

When context is tensor([18]), target is 47
When context is tensor([18, 47]), target is 56
When context is tensor([18, 47, 56]), target is 57
When context is tensor([18, 47, 56, 57]), target is 58
When context is tensor([18, 47, 56, 57, 58]), target is 1
When context is tensor([18, 47, 56, 57, 58,  1]), target is 15
When context is tensor([18, 47, 56, 57, 58,  1, 15]), target is 47
When context is tensor([18, 47, 56, 57, 58,  1, 15, 47]), target is 58


In [49]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8


def get_batch(split):
    """
    Get one batch of x, y
    Input : "train", else assumes val data
    Samples a random index b/w start of data and end (till :-block_size) 
    and outputs batch_size slices of indices of block size - so 4,8 here
    both x and y will be of that size
    """
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,)) #random indice arrays upto a max of size-block size, do this batch size times
    x = torch.stack([data[i:i+block_size] for i in ix]) 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y


In [50]:
xb, yb = get_batch("train")
print('inputs ', xb, '\n  targets', yb)

inputs  tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]]) 
  targets tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])


In [51]:
for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, :t+1]
        target = yb[b, t] #note that yb is already right shifted, so the target will be at index t
        print(f"When context is {context.tolist()}, target is {target}")

When context is [24], target is 43
When context is [24, 43], target is 58
When context is [24, 43, 58], target is 5
When context is [24, 43, 58, 5], target is 57
When context is [24, 43, 58, 5, 57], target is 1
When context is [24, 43, 58, 5, 57, 1], target is 46
When context is [24, 43, 58, 5, 57, 1, 46], target is 43
When context is [24, 43, 58, 5, 57, 1, 46, 43], target is 39
When context is [44], target is 53
When context is [44, 53], target is 56
When context is [44, 53, 56], target is 1
When context is [44, 53, 56, 1], target is 58
When context is [44, 53, 56, 1, 58], target is 46
When context is [44, 53, 56, 1, 58, 46], target is 39
When context is [44, 53, 56, 1, 58, 46, 39], target is 58
When context is [44, 53, 56, 1, 58, 46, 39, 58], target is 1
When context is [52], target is 58
When context is [52, 58], target is 1
When context is [52, 58, 1], target is 58
When context is [52, 58, 1, 58], target is 46
When context is [52, 58, 1, 58, 46], target is 39
When context is [52, 5

In [52]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets):
        logits = self.token_embedding_table(idx) #(B, T, C)

        return logits
    
m = BigramLanguageModel(vocab_size)
    

In [53]:
out = m(xb, yb)
print(out, out.shape)

tensor([[[-1.5101, -0.0948,  1.0927,  ..., -0.6126, -0.6597,  0.7624],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305],
         ...,
         [ 0.5978, -0.0514, -0.0646,  ..., -1.4649, -2.0555,  1.8275],
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 0.3323, -0.0872, -0.7470,  ..., -0.6716, -0.9572, -0.9594]],

        [[ 1.0541,  1.5018, -0.5266,  ...,  1.8574,  1.5249,  1.3035],
         [-0.1324, -0.5489,  0.1024,  ..., -0.8599, -1.6050, -0.6985],
         [-0.6722,  0.2322, -0.1632,  ...,  0.1390,  0.7560,  0.4296],
         ...,
         [ 1.0901,  0.2170, -2.9996,  ..., -0.5472, -0.8017,  0.7761],
         [ 1.1513,  1.0539,  3.4105,  ..., -0.5686,  0.9079, -0.1701],
         [ 0.2475, -0.6349, -1.2909,  ...,  1.3064, -0.2256, -1.8305]],

        [[-0.2103,  0.4481,  1.2381,  ...,  1.3597, -0.0821,  0.3909],
         [ 0.2475, -0.6349, -1.2909,  ...,  1